## Import parqet files

In [1]:
# Load data from parquet file
import pandas as pd
import numpy as np
from project_1.config import PROJ_ROOT, PROCESSED_DATA_DIR

sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}/set_{set_name}_final.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Assure the loading was correct
print(sets_dict["set_a"].shape)
print(type(sets_dict["set_a"]))
sets_dict["set_a"].head(10)


2025-03-27 17:05:15.470 | INFO     | project_1.config:<module>:11 - PROJ_ROOT path is: C:\Users\tilma\OneDrive\Studium\_Machine Learning for Healthcare\Projects


(183416, 43)
<class 'pandas.core.frame.DataFrame'>


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,ICUType,K,Lactate,MechVent,Urine,WBC,pH,SaO2,TroponinT,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-0.953317,0.283445,-0.596332,1.004449,-0.069047,-0.739159,-1.269243,...,0.5,0.625,-0.583333,0.0,2.96,0.271429,-0.750,1.0,0.933333,-0.224490
1,132539.0,2025-03-10 01:00:00,0.0,-0.953317,0.283445,-0.596332,-0.232975,1.997003,0.232700,-2.924060,...,0.5,0.375,-0.166667,0.0,3.20,0.000000,0.750,-15.5,-0.177778,0.408163
2,132539.0,2025-03-10 02:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,2.628972,2.440210,0.385573,...,0.5,0.625,2.000000,0.0,-0.40,2.185714,-0.250,1.0,0.333333,-0.285714
3,132539.0,2025-03-10 03:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,-1.721888,1.169851,-0.441835,...,0.5,-1.000,-0.416667,0.0,0.72,2.185714,-0.250,0.5,0.333333,2.040816
4,132539.0,2025-03-10 04:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,1.462261,-0.392067,-2.510356,...,0.5,0.125,0.000000,0.0,-0.16,-0.957143,0.375,1.0,0.200000,0.795918
5,132539.0,2025-03-10 05:00:00,0.0,-0.953317,0.283445,-0.596332,1.341928,1.146276,-0.235875,0.178721,...,0.5,-2.125,1.666667,0.0,-0.16,1.557143,-0.375,0.0,-0.244444,0.183673
6,132539.0,2025-03-10 06:00:00,0.0,-0.953317,0.283445,-0.596332,1.341928,-0.506564,-0.322648,-0.234983,...,0.5,0.625,0.833333,0.0,0.72,1.557143,-0.125,0.0,0.800000,-0.081633
7,132539.0,2025-03-10 08:00:00,0.0,-0.953317,0.283445,-0.596332,-1.357905,-0.117660,-0.010265,1.419833,...,0.5,-0.250,0.250000,0.0,0.32,-0.657143,-0.625,0.0,20.911111,8.408163
8,132539.0,2025-03-10 09:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,0.344163,0.232700,-0.028131,...,0.5,-0.250,-0.416667,0.0,0.00,-0.657143,-0.250,0.0,-0.088889,-0.020408
9,132539.0,2025-03-10 10:00:00,0.0,-0.953317,0.283445,-0.596332,-1.470399,0.076791,0.302119,0.385573,...,0.5,-0.375,4.000000,0.0,0.16,-0.014286,-1.000,0.0,1.555556,0.040816


In [3]:
df_a = sets_dict["set_a"]
print(df_a.describe())
print(df_a.columns)

            RecordID                           Time         Gender  \
count  183416.000000                         183416  183416.000000   
mean   137604.313779  2025-03-10 23:59:34.798273280       0.562081   
min    132539.000000            2025-03-10 00:00:00       0.000000   
25%    135072.000000            2025-03-10 12:00:00       0.000000   
50%    137590.000000            2025-03-11 00:00:00       1.000000   
75%    140101.000000            2025-03-11 12:00:00       1.000000   
max    142673.000000            2025-03-12 00:00:00       1.000000   
std      2923.639911                            NaN       0.496002   

             Height        Weight           Age       Albumin   Cholesterol  \
count  1.834160e+05  1.834160e+05  1.834160e+05  1.834160e+05  1.834160e+05   
mean   6.454295e-15  4.456582e-16 -2.810931e-16 -5.493248e-16  2.112266e-16   
min   -5.158858e+00 -2.611045e+00 -2.835966e+00 -2.482836e+00 -3.083051e+00   
25%   -7.575847e-01 -6.533555e-01 -6.537587e-01 -7.95

# Q4.1 Prompting an LLM to solve a time-series problem

## Patient Summary short
only includes the most important data, without much noise

In [4]:
def generate_summary_statistical(patient_df):
    features = {}
    # Assuming df is already filtered for a single patient
    features['age'] = patient_df['Age'].iloc[0]
    features['gender'] = "male" if patient_df['Gender'].iloc[0] == 1 else "female"

    summary = [
        f"Patient is a {int(features['age']*40+60)}-year-old {features['gender']}. Over the first 48 hours:"
    ]

    vital_vars = ['HeartRate', 'SaO2', 'WBC', 'Urine', 'pH', 'Lactate']
    for var in vital_vars:
        if var not in patient_df.columns: continue
        values = patient_df[var].dropna()
        if len(values) == 0: continue
        min_val, max_val = values.min(), values.max()
        mean_val, last_val = values.iloc[-1], values.iloc[-1]
        summary.append(f"- {var} ranged from {min_val:.2f} to {max_val:.2f}, avg: {mean_val:.2f}, last: {last_val:.2f}")

    summary.append("Predict the risk of in-hospital death on a scale from 1 to 10.")
    return "\n".join(summary)


## Patient Summary trends
Summarize variable behavior over three time segments (0–16h, 16–32h, 32–48h) using trends: increasing, decreasing, stable, or fluctuating. Adds a temporal element that can help the LLM infer progression.

In [5]:
def get_trend_label(seq):
    if len(seq) < 2: return "unknown"
    d1, d2, d3 = np.mean(seq[:len(seq)//3]), np.mean(seq[len(seq)//3:2*len(seq)//3]), np.mean(seq[2*len(seq)//3:])
    if d1 < d2 < d3: return "↑"
    elif d1 > d2 > d3: return "↓"
    elif abs(d1 - d2) < 0.1 and abs(d2 - d3) < 0.1: return "→"
    else: return "~"

def generate_summary_trend(patient_df):
    features = {}
    features['age'] = patient_df['Age'].iloc[0]
    features['gender'] = "male" if patient_df['Gender'].iloc[0] == 1 else "female"

    summary = [
        f"Patient is a {int(features['age']*40+60)}-year-old {features['gender']}. Over the first 48 hours:"
    ]

    vital_vars = ['HeartRate', 'SaO2', 'WBC', 'Urine', 'pH', 'Lactate']
    for var in vital_vars:
        if var not in patient_df.columns: continue
        series = patient_df[var].dropna().values
        if len(series) < 3: continue
        trend = get_trend_label(series)
        summary.append(f"- {var} shows a trend of {trend}")

    summary.append("Predict the patient's in-hospital death risk.")
    return "\n".join(summary)


In [7]:
def generate_patient_summaries(df):
    result_rows = []

    for rid, group in df.groupby("RecordID"):
        summary_stat = generate_summary_statistical(group)
        summary_trend = generate_summary_trend(group)

        result_rows.append({
            "RecordID": rid,
            "summary_statistical": summary_stat,
            "summary_trend": summary_trend
        })

    return pd.DataFrame(result_rows)

# ---------- Usage ---------- #

# Assuming df_a is your full DataFrame
summary_df = generate_patient_summaries(df_a)

# Show a few example summaries
summary_df.head(2)

,RecordID,summary_statistical,summary_trend
0,132539.0,Patient is a 36-year-old female. Over the firs...,Patient is a 36-year-old female. Over the firs...
1,132540.0,Patient is a 86-year-old male. Over the first ...,Patient is a 86-year-old male. Over the first ...


In [11]:
(summary_df.head(1)["summary_trend"])

0    Patient is a 36-year-old female. Over the firs...
Name: summary_trend, dtype: object

### Ollama Test

In [12]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='gemma3:1b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
# print(response.message.content)

The sky is blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown of why it happens:

* **Sunlight is made of all colors:**  Sunlight appears white to us, but it's actually composed of all the colors of the rainbow – red, orange, yellow, green, blue, indigo, and violet.

* **Entering the atmosphere:** As sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).

* **Rayleigh scattering:**  This collision causes the light to scatter in different directions.  **Rayleigh scattering is much more effective at shorter wavelengths of light, like blue and violet.**  Think of it like this: smaller particles scatter light more easily than larger ones.

* **Why blue and not violet?** Violet light is scattered even more than blue, but our eyes are more sensitive to blue light, and the sun emits slightly less violet.  Also, some violet is absorbed by the upper atmosphere.

**In simpler terms:** Imagine throwing a handful of bouncy

### Result Calculation

In [13]:
from ollama import chat
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np
import pandas as pd
import re

# ---- Utility: Extract score or binary ---- #

def extract_score_from_response(text, mode="score"):
    """
    Extracts a number from model output or interprets binary label.
    """
    if mode == "score":
        match = re.search(r'(\d+(\.\d+)?)', text)
        if match:
            val = float(match.group(1))
            return min(max(val, 1.0), 10.0) / 10.0  # normalize 1-10 to 0-1
    else:  # binary mode
        text = text.lower()
        if any(word in text for word in ["yes", "high", "die", "likely"]):
            return 1.0
        elif any(word in text for word in ["no", "low", "survive", "unlikely"]):
            return 0.0
    return 0.5  # fallback neutral

# ---- Few-shot formatter ---- #

def format_few_shot_examples(df_train, label_col, text_col, max_examples=3, mode="score"):
    """
    Creates few-shot context examples from labeled training set.
    """
    examples = df_train.sample(n=max_examples, random_state=42)
    formatted = ""
    for _, row in examples.iterrows():
        label = row[label_col]
        label_text = f"{int(label * 9 + 1)}/10" if mode == "score" else ("Yes" if label == 1 else "No")
        formatted += f"Input:\n{row[text_col]}\nAnswer: {label_text}\n\n"
    return formatted.strip()

# ---- Core inference function ---- #

def query_llm_gemma(summary_text, few_shot_context=None):
    """
    Calls gemma3:1b model via ollama.chat using user prompt.
    """
    prompt = f"{few_shot_context}\n\nInput:\n{summary_text}\nAnswer:" if few_shot_context else f"Input:\n{summary_text}\nAnswer:"
    
    response = chat(model='gemma3:1b', messages=[{
        'role': 'user',
        'content': prompt
    }])
    
    return response['message']['content']

# ---- Full evaluation runner ---- #

def evaluate_llm_predictions(summary_df, label_col, text_col='summary_statistical', mode="score",
                              few_shot=False, df_train_for_examples=None):
    """
    Runs inference across all rows in summary_df and returns performance + predictions.
    """
    predictions = []
    labels = summary_df[label_col].values

    # Build few-shot context once
    context = None
    if few_shot and df_train_for_examples is not None:
        context = format_few_shot_examples(df_train_for_examples, label_col, text_col, max_examples=3, mode=mode)

    for _, row in summary_df.iterrows():
        summary = row[text_col]
        llm_output = query_llm_gemma(summary, few_shot_context=context)
        score = extract_score_from_response(llm_output, mode=mode)
        predictions.append(score)

    predictions = np.array(predictions)

    try:
        auroc = roc_auc_score(labels, predictions)
        auprc = average_precision_score(labels, predictions)
    except:
        auroc = None
        auprc = None

    return {
        "predictions": predictions,
        "true_labels": labels,
        "auroc": auroc,
        "auprc": auprc
    }


### Zero-Shot, Score-Mode

In [14]:
results = evaluate_llm_predictions(
    summary_df,
    label_col='label',
    text_col='summary_statistical',
    mode='score'
)
print("Zero-shot score mode →", results["auroc"], results["auprc"])


KeyError: 'label'

### Zero-Shot, Binary-Mode

In [ ]:
results = evaluate_llm_predictions(
    summary_df,
    label_col='label',
    text_col='summary_trend',
    mode='binary'
)
print("Zero-shot binary mode →", results["auroc"], results["auprc"])


### Few-Shot, Score-Mode

In [ ]:
results = evaluate_llm_predictions(
    summary_df,
    label_col='label',
    text_col='summary_statistical',
    mode='score',
    few_shot=True,
    df_train_for_examples=train_df  # Must be labeled
)
print("Few-shot score mode →", results["auroc"], results["auprc"])


# Q4.2 Using LLs to retrieve embeddings

In [ ]:
import numpy as np
import pandas as pd
from ollama import embedding
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def get_embedding_ollama(text: str, model_name='gemma3:1b'):
    """
    Calls the Ollama embedding API for a given string.
    """
    response = embedding(model=model_name, prompt=text)
    return np.array(response.get("embedding", []), dtype=float)

def build_embeddings(df, summary_col="summary_statistical", model_name='gemma3:1b'):
    """
    For each row in the DataFrame, get an embedding and corresponding label.
    """
    embeddings = []
    labels = []

    for _, row in df.iterrows():
        text = row[summary_col]
        label = row["label"]
        emb = get_embedding_ollama(text, model_name=model_name)
        embeddings.append(emb)
        labels.append(label)

    return np.vstack(embeddings), np.array(labels)

def train_and_evaluate(train_X, train_y, test_X, test_y):
    clf = LogisticRegression(max_iter=2000)
    clf.fit(train_X, train_y)
    probs = clf.predict_proba(test_X)[:, 1]

    auroc = roc_auc_score(test_y, probs)
    auprc = average_precision_score(test_y, probs)
    return probs, auroc, auprc

def visualize_tsne(embeddings, labels, title="t-SNE of Embeddings"):
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    emb_2d = tsne.fit_transform(embeddings)

    plt.figure(figsize=(8,6))
    scatter = plt.scatter(emb_2d[:,0], emb_2d[:,1], c=labels, cmap='coolwarm', alpha=0.7)
    plt.colorbar(scatter, label="In-hospital Death (0/1)")
    plt.title(title)
    plt.xlabel("TSNE-1")
    plt.ylabel("TSNE-2")
    plt.show()


In [ ]:
# Replace df_train and df_test with your actual DataFrames

train_X, train_y = build_embeddings(df_train, summary_col="summary_statistical", model_name='gemma3:1b')
test_X, test_y = build_embeddings(df_test, summary_col="summary_statistical", model_name='gemma3:1b')

probs, auroc, auprc = train_and_evaluate(train_X, train_y, test_X, test_y)
print(f"Test AuROC: {auroc:.4f}, Test AuPRC: {auprc:.4f}")

visualize_tsne(test_X, test_y, title="t-SNE of gemma3:1b Embeddings (Test Set)")


# Q4.3 Using time-series foundation models

In [15]:
# If not installed yet:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to c:\users\tilma\appdata\local\temp\pip-req-build-h6u10xk3
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 94e20ea7e510ac4d665492b8bed8836a5143f16e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.2 MB 8.3 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.2 MB 11.8 MB/s eta 0:00:01
   ------------------ --------------------- 4.7/10.2 MB 13.0 MB/s eta 0:00:01
   ------------------------- -------------- 6.6/10.2 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------

  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git 'C:\Users\tilma\AppData\Local\Temp\pip-req-build-h6u10xk3'

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

import pandas as pd
import torch
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

df = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")

# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
context = torch.tensor(df["#Passengers"])
embeddings, tokenizer_state = pipeline.embed(context)

# 1) Single Embedding + Linear Probe
train_X, train_y = build_patient_embeddings_naive(pretrained_model, df_train)
test_X, test_y = build_patient_embeddings_naive(pretrained_model, df_test)

clf = LogisticRegression(max_iter=2000)
clf.fit(train_X, train_y)
probs = clf.predict_proba(test_X)[:,1]
auroc = roc_auc_score(test_y, probs)
auprc = average_precision_score(test_y, probs)
print(f"[Naive Aggregation] AuROC={auroc:.4f}, AuPRC={auprc:.4f}")

Making a smarter aggregation: 
Instead of simply averaging embeddings across variables, we can:

Compute an embedding for each variable.

Feed all variable embeddings into a small MLP that outputs a patient-level embedding.

Then, either attach a logistic head for classification or just produce a final embedding for a standard logistic regression.

In [ ]:

########################################
# 2) Simple Neural Aggregator Approach #
########################################

class ChannelAggregator(nn.Module):
    def __init__(self, embed_dim=128, hidden_dim=64, out_dim=1):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(embed_dim*10, hidden_dim),  # if you expect ~10 variables
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )
    
    def forward(self, x):
        return self.mlp(x)

def build_multivar_embeddings(model, multivar_data):
    embeddings = []
    for var_name, series in multivar_data.items():
        emb = get_univariate_embedding(model, series)
        embeddings.append(emb.unsqueeze(0))
    if len(embeddings) == 0:
        return torch.zeros((1, 128))
    return torch.cat(embeddings, dim=0)

def train_aggregator(pretrained_model, aggregator, df_patients, epochs=5, lr=1e-3, device='cpu'):
    aggregator.to(device)
    optimizer = torch.optim.Adam(aggregator.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    # Build full dataset in memory
    X_list, y_list = [], []
    for _, row in df_patients.iterrows():
        data_dict = row['multivar_data']
        emb_stack = build_multivar_embeddings(pretrained_model, data_dict)
        X_list.append(emb_stack.unsqueeze(0)) # shape [1, num_vars, embed_dim]
        y_list.append(row['label'])
    
    X_tensor = torch.cat(X_list, dim=0).float().to(device)
    y_tensor = torch.tensor(y_list, dtype=torch.float).unsqueeze(-1).to(device)

    for epoch in range(epochs):
        aggregator.train()
        optimizer.zero_grad()
        logits = aggregator(X_tensor)
        loss = criterion(logits, y_tensor)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")
    
    return aggregator

def evaluate_aggregator(pretrained_model, aggregator, df_patients, device='cpu'):
    aggregator.eval()
    all_logits = []
    all_labels = []

    with torch.no_grad():
        for _, row in df_patients.iterrows():
            data_dict = row['multivar_data']
            emb_stack = build_multivar_embeddings(pretrained_model, data_dict).unsqueeze(0).to(device)
            logit = aggregator(emb_stack)
            all_logits.append(logit.item())
            all_labels.append(row['label'])
    
    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)
    probs = 1 / (1 + np.exp(-all_logits))

    auroc = roc_auc_score(all_labels, probs)
    auprc = average_precision_score(all_labels, probs)
    return auroc, auprc

aggregator = ChannelAggregator(embed_dim=128, hidden_dim=64, out_dim=1)
aggregator = train_aggregator(pretrained_model, aggregator, df_train, epochs=5, lr=1e-3, device='cpu')
auroc, auprc = evaluate_aggregator(pretrained_model, aggregator, df_test)
print(f"[Neural Aggregator] AuROC={auroc:.4f}, AuPRC={auprc:.4f}")